In [1]:
from profile_generator import generate_profiles

In [2]:
n = 50
topics = [
    'Business',
    'Entertainment',
    'Politics',
    'Sports',
    'Technology',
]

profiles = generate_profiles(n, topics)
profiles

{0: ['Entertainment'],
 1: ['Politics', 'Entertainment', 'Sports', 'Technology'],
 2: ['Sports', 'Technology', 'Politics', 'Business', 'Entertainment'],
 3: ['Technology'],
 4: ['Entertainment', 'Sports', 'Politics', 'Business'],
 5: ['Politics', 'Sports'],
 6: ['Entertainment', 'Technology', 'Business'],
 7: ['Politics', 'Entertainment', 'Business', 'Sports'],
 8: ['Technology', 'Entertainment', 'Business'],
 9: ['Entertainment', 'Sports', 'Business', 'Technology'],
 10: ['Entertainment', 'Business', 'Sports'],
 11: ['Entertainment', 'Sports', 'Politics', 'Technology'],
 12: ['Sports'],
 13: ['Business', 'Sports', 'Entertainment', 'Politics', 'Technology'],
 14: ['Entertainment', 'Sports', 'Politics', 'Business'],
 15: ['Sports'],
 16: ['Politics', 'Sports'],
 17: ['Politics', 'Sports', 'Entertainment', 'Business', 'Technology'],
 18: ['Sports', 'Entertainment', 'Politics', 'Business', 'Technology'],
 19: ['Politics', 'Technology', 'Entertainment', 'Business'],
 20: ['Entertainment', 